In [1]:
# coding: utf-8

'''
This module is formatting  user config with facts and entities,
creating entity index and producing all variants for entity-match
'''

import zipfile
import gensim

import re
import os
from tqdm import tqdm
from operator import itemgetter
from scipy import stats
import random
from functools import lru_cache
import pandas as pd
import dill as pickle

In [2]:
def open_model(m):
    try:
        if m.endswith('.vec'):
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
        elif m.endswith('.bin'):
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

        else:
            model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False, unicode_errors='replace')
    except:
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False, unicode_errors='replace')
    return model

In [3]:
# common words
f = open(r'/media/mi_air/0F0B7DDE62EEA81E/vector/intersec_words_english.txt','r', encoding='utf8').read().split('\n')
f

['embroider_VERB',
 'Takeda_PROPN',
 'autocracy_NOUN',
 'nimble_ADJ',
 'Ursus_PROPN',
 'df_NOUN',
 'protozoa_NOUN',
 'over-the-counter_ADJ',
 'Aidid_PROPN',
 'Rajya::Sabha_PROPN',
 'lengthening_NOUN',
 'bump_NOUN',
 'addressable_ADJ',
 'cardiologist_NOUN',
 'wily_ADJ',
 'Woodrow::Wilson_PROPN',
 'equaliser_NOUN',
 'Irina_PROPN',
 'wide-screen_ADJ',
 'pitfall_NOUN',
 'now-familiar_ADJ',
 'Bloomingdales_PROPN',
 'indicate_VERB',
 'reintegrate_VERB',
 'Haiti_PROPN',
 '6-3_NUM',
 'standardized_ADJ',
 'Vorarlberg_PROPN',
 'outspokenness_NOUN',
 'intentional_ADJ',
 'agonizingly_ADV',
 'jun_NOUN',
 'aroma_NOUN',
 'spirit_VERB',
 'speechwriter_NOUN',
 'enfold_VERB',
 'Amur_PROPN',
 'poodle_NOUN',
 'guiltless_ADJ',
 'trend_VERB',
 'tu_X',
 'Mulcahy_PROPN',
 'transliteration_NOUN',
 'waist-deep_ADJ',
 'properly_ADV',
 'Mullah_PROPN',
 'Mears_PROPN',
 'stitching_NOUN',
 'aspiration_NOUN',
 'corn_NOUN',
 '20th_ADJ',
 'saboteur_NOUN',
 'long-forgotten_ADJ',
 'snorkel_NOUN',
 'Cindy::Sherman_PROPN',

In [4]:
dic_all_words = { 'bnc':{},'wiki':{}, 'gigaword':{}}

In [5]:

WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector/'

#bnc
with zipfile.ZipFile(WDIR + "bnc_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [6]:
num = 100

In [7]:
for w in tqdm(f):
    if w not in dic_all_words['bnc']:
        dic_all_words['bnc'][w] = model.most_similar(w,topn=num)

  0%|          | 0/60227 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 60227/60227 [25:34<00:00, 39.24it/s]


In [8]:
with zipfile.ZipFile(WDIR + "gigaword_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [9]:
for w in tqdm(f):
    if w not in dic_all_words['gigaword']:
        dic_all_words['gigaword'][w] = model.most_similar(w,topn=num)

  0%|          | 0/60227 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 60227/60227 [44:07<00:00, 22.75it/s] 


In [10]:
#wiki
with zipfile.ZipFile(WDIR + "wiki_en.zip", "r") as archive:
    stream = archive.open("model.txt")
model = open_model(stream)

In [11]:
for w in tqdm(f):
    if w not in dic_all_words['wiki']:
        dic_all_words['wiki'][w] = model.most_similar(w,topn=num)

  0%|          | 0/60227 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 60227/60227 [43:24<00:00, 16.84it/s] 


In [12]:
model=[]
stream=0

In [13]:
# теперь найдем пересечение

def get_words(dic):
    return [i[0] for i in dic]

def count_set(list1, list2, list3):
    set1, set2, set3 = set(list1), set(list2), set(list3)
    d = set.intersection(set1, set2, set3)
    u = set(list(set1)+list(set2)+list(set3))
    return len(d)/len(u)



rows_list = []
for word in tqdm(f):
    dict1 = {}
    dict1['word'] = word
    dict1['svodesh/random'] = 'all'
    dict1['% intersection'] = count_set(get_words(dic_all_words['bnc'][word]),get_words(dic_all_words['gigaword'][word]),get_words(dic_all_words['wiki'][word]))
    rows_list.append(dict1)

100%|██████████| 60227/60227 [00:13<00:00, 4516.72it/s]


In [14]:
df100 = pd.DataFrame(rows_list) 
rows_list=[]
df100.head()

,% intersection,svodesh/random,word
0,0.021008,all,embroider_VERB
1,0.000000,all,Takeda_PROPN
2,0.020492,all,autocracy_NOUN
3,0.035857,all,nimble_ADJ
4,0.000000,all,Ursus_PROPN


In [16]:
df100 = df100.sort_values(['% intersection'], ascending=False)

In [25]:
df100clear.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/eng_all_word_intersection_100.csv', sep='\t')

In [17]:
df100.head(100)

,% intersection,svodesh/random,word
55808,0.735043,all,forty-two_NUM
20172,0.720339,all,twenty-seven_NUM
29639,0.705882,all,twenty-eight_NUM
47525,0.705882,all,twenty-two_NUM
55276,0.700000,all,twenty-three_NUM
37392,0.700000,all,thirty-eight_NUM
39537,0.697479,all,twenty-six_NUM
3146,0.694215,all,thirty-seven_NUM
22059,0.691667,all,twenty-five_NUM
59192,0.683333,all,thirty-six_NUM


In [24]:
df100clear = []
for i in range(len(df100)):
    if 'NUM' not in df100.iloc[i]['word'] and not re.search('\d+', df100.iloc[i]['word']):
        df100clear.append(df100.iloc[i])
df100clear =  pd.DataFrame(df100clear) 
df100clear.head()

,% intersection,svodesh/random,word
54777,0.265896,all,six-month_ADJ
37109,0.238889,all,three-week_ADJ
23429,0.213115,all,two-week_ADJ
29847,0.193548,all,eight-year_ADJ
20880,0.191710,all,six-week_ADJ


In [26]:
top10verbs = []
j=0
for i in range(len(df100clear)):
    if 'VERB' in df100clear.iloc[i]['word']:
        top10verbs.append(df100clear.iloc[i]['word'])
        j+=1
    if j>10:
        break

In [27]:
top10verbs

['deny_VERB',
 'convict_VERB',
 'restore_VERB',
 'recycle_VERB',
 'tend_VERB',
 'remarry_VERB',
 'acquit_VERB',
 'damage_VERB',
 'fracture_VERB',
 'reject_VERB',
 'abuse_VERB']

In [28]:
dfrus = pd.read_csv('/media/mi_air/0F0B7DDE62EEA81E/vector/all_word_intersection_100.csv', sep='\t')
top10verbs = []
j=0
for i in range(len(dfrus)):
    if 'VERB' in dfrus.iloc[i]['word']:
        top10verbs.append(dfrus.iloc[i]['word'])
        j+=1
    if j>10:
        break
top10verbs

['рычать_VERB',
 'рыть_VERB',
 'разъезжать_VERB',
 'закричать_VERB',
 'завопить_VERB',
 'сокрушаться_VERB',
 'расширять_VERB',
 'ссориться_VERB',
 'пугать_VERB',
 'жаловаться_VERB',
 'ругать_VERB']